In [1]:
!python --version

Python 3.10.12


In [2]:
import torch
torch.__version__

'2.5.1+cu121'

In [3]:
import matplotlib.pyplot as plt
import torch
import torchvision

from torch import nn
from torchvision import transforms

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [5]:
# Install required libraries
!pip install thop
!pip install torchsummary
!pip install albumentations

import numpy as np
import pandas as pd
import os
import shutil
from shutil import copy, rmtree
import tensorflow as tf
import cv2
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from PIL import Image
import albumentations as A
from albumentations.pytorch import ToTensorV2

/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 2.0.5 (you have 1.4.20). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [6]:
# Load metadata
data_pd = pd.read_csv('/kaggle/input/skin-cancer-mnist-ham10000/HAM10000_metadata.csv')
print(data_pd.head())

     lesion_id      image_id   dx dx_type   age   sex localization
0  HAM_0000118  ISIC_0027419  bkl   histo  80.0  male        scalp
1  HAM_0000118  ISIC_0025030  bkl   histo  80.0  male        scalp
2  HAM_0002730  ISIC_0026769  bkl   histo  80.0  male        scalp
3  HAM_0002730  ISIC_0025661  bkl   histo  80.0  male        scalp
4  HAM_0001466  ISIC_0031633  bkl   histo  75.0  male          ear


In [7]:
df_count = data_pd.groupby('lesion_id').count()
df_count.head()

,image_id,dx,dx_type,age,sex,localization
lesion_id,,,,,,
HAM_0000000,2,2,2,2,2,2
HAM_0000001,1,1,1,1,1,1
HAM_0000002,3,3,3,3,3,3
HAM_0000003,1,1,1,1,1,1
HAM_0000004,1,1,1,1,1,1


In [8]:
df_count = df_count[df_count['dx'] == 1]
df_count.reset_index(inplace=True)

In [9]:
def duplicates(x):
    unique = set(df_count['lesion_id'])
    if x in unique:
        return 'no' 
    else:
        return 'duplicates'

In [10]:
data_pd['is_duplicate'] = data_pd['lesion_id'].apply(duplicates)
data_pd.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization,is_duplicate
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp,duplicates
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp,duplicates
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp,duplicates
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp,duplicates
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear,duplicates


In [11]:
df_count = data_pd[data_pd['is_duplicate'] == 'no']

In [12]:
# Paths to the dataset
image_dir_part1 = '/kaggle/input/skin-cancer-mnist-ham10000/HAM10000_images_part_1'
image_dir_part2 = '/kaggle/input/skin-cancer-mnist-ham10000/HAM10000_images_part_2'
csv_file = '/kaggle/input/skin-cancer-mnist-ham10000/HAM10000_metadata.csv'

# Reading the metadata CSV
df = pd.read_csv(csv_file)

# Create the root directory where we will save train, test, and val sets
output_dir = '/kaggle/working/HAM10000_split'
train_dir = os.path.join(output_dir, 'train')
val_dir = os.path.join(output_dir, 'val')
test_dir = os.path.join(output_dir, 'test')

# Create folders for train, val, and test sets
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Split data into train, test, and validation sets
train_val_df, test_df = train_test_split(df, test_size=0.15, stratify=df['dx'], random_state=42)
train_df, val_df = train_test_split(train_val_df, test_size=0.15, stratify=train_val_df['dx'], random_state=42)

# Function to copy images to their respective folders
def copy_images(image_list, part1_dir, part2_dir, destination_dir):
    for _, row in image_list.iterrows():
        file_name = f"{row['image_id']}.jpg"
        src_part1 = os.path.join(part1_dir, file_name)
        src_part2 = os.path.join(part2_dir, file_name)
        
        # Check if the image is in part 1 or part 2
        if os.path.exists(src_part1):
            src = src_part1
        elif os.path.exists(src_part2):
            src = src_part2
        else:
            print(f"File not found: {file_name}")
            continue
        
        dst = os.path.join(destination_dir, row['dx'])
        os.makedirs(dst, exist_ok=True)  # Create class subfolder if it doesn't exist
        shutil.copy(src, dst)

# Copy images into train, val, and test folders
copy_images(train_df, image_dir_part1, image_dir_part2, train_dir)
copy_images(val_df, image_dir_part1, image_dir_part2, val_dir)
copy_images(test_df, image_dir_part1, image_dir_part2, test_dir)

print(f"Train set: {len(train_df)} images")
print(f"Validation set: {len(val_df)} images")
print(f"Test set: {len(test_df)} images")

Train set: 7235 images
Validation set: 1277 images
Test set: 1503 images


In [13]:
import os
from shutil import copy, rmtree
import torch
import torchvision.transforms as transforms
from PIL import Image
import albumentations as A
from albumentations.pytorch import ToTensorV2

# Define paths
origin_data_path = '/kaggle/working/HAM10000_split/train'  # Update path

# Get class names (subdirectories)
data_class = [cla for cla in os.listdir(origin_data_path)
              if os.path.isdir(os.path.join(origin_data_path, cla))]

# Define augmentations (Optimized for ViT-Base-16)
augmentations = A.Compose([
    A.RandomResizedCrop(224, 224, scale=(0.8, 1.0)),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    A.HueSaturationValue(p=0.2),
    A.Rotate(limit=15, p=0.5),
    A.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
    ToTensorV2()
])

# Augment images for each class
for img_class in data_class:
    class_path = os.path.join(origin_data_path, img_class)
    img_list = os.listdir(class_path)
    
    # Create temporary directory
    aug_dir = "aug_dir"
    img_dir = os.path.join(aug_dir, img_class)
    os.makedirs(img_dir, exist_ok=True)

    # Copy original images
    for image in img_list:
        image_path = os.path.join(class_path, image)
        if os.path.isfile(image_path):
            copy(image_path, os.path.join(img_dir, image))

    aug_images = 8000  # Target number of augmented images
    num_files = len(img_list)
    extra_images_needed = aug_images - num_files

    # Apply augmentation
    for i in range(extra_images_needed):
        img_name = img_list[i % num_files]  # Cycle through existing images
        img_path = os.path.join(class_path, img_name)

        image = Image.open(img_path).convert('RGB')
        image = np.array(image)
        
        # Apply Albumentations augmentation
        augmented = augmentations(image=image)['image']
        
        # Save augmented image
        aug_img_name = f"trans_{i}.jpg"
        aug_img_path = os.path.join(class_path, aug_img_name)
        augmented_pil = transforms.ToPILImage()(augmented)
        augmented_pil.save(aug_img_path)

    # Remove temporary directory
    rmtree(aug_dir)

print("Augmentation completed successfully.")

Augmentation completed successfully.


In [14]:
# detect 
total_num = 0
for cla in data_class:
    cla_path = os.path.join('/kaggle/working/HAM10000_split/train', cla)
    images = os.listdir(cla_path)
    num = len(images)
    total_num += num
    for index, image in enumerate(images):
 
        print("\r[{}] processing [{}/{}]".format(cla, index+1, num), end="")  # processing bar
    # break
    print()

print(f"processing {total_num} done!")

[df] processing [8000/8000]
[mel] processing [8000/8000]
[vasc] processing [8000/8000]
[nv] processing [8000/8000]
[bkl] processing [8000/8000]
[akiec] processing [8000/8000]
[bcc] processing [8000/8000]
processing 56000 done!


In [15]:
# detect 
total_num = 0
for cla in data_class:
    cla_path = os.path.join('/kaggle/working/HAM10000_split/test', cla)
    images = os.listdir(cla_path)
    num = len(images)
    total_num += num
    for index, image in enumerate(images):
 
        print("\r[{}] processing [{}/{}]".format(cla, index+1, num), end="")  # processing bar
    # break
    print()

print(f"processing {total_num} done!")

[df] processing [17/17]
[mel] processing [167/167]
[vasc] processing [22/22]
[nv] processing [1006/1006]
[bkl] processing [165/165]
[akiec] processing [49/49]
[bcc] processing [77/77]
processing 1503 done!


In [16]:
import random
import numpy as np
import torch

def set_seeds(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


In [17]:
# Set random seeds for reproducibility
set_seeds()


In [18]:
import torch
import torch.nn as nn
import torchvision
from torchinfo import summary

# Set device
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# 1. Get pretrained weights for ViT-Base
pretrained_vit_weights = torchvision.models.ViT_B_16_Weights.DEFAULT 

# 2. Setup a ViT model instance with pretrained weights
pretrained_vit = torchvision.models.vit_b_16(weights=pretrained_vit_weights).to(device)

# 3. Freeze the base parameters
for parameter in pretrained_vit.parameters():
    parameter.requires_grad = False

# 4. Change the classifier head for the HAM10000 dataset (7 classes)
class_names = ['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'vasc']  # HAM10000 dataset classes

# Set random seeds for reproducibility
set_seeds()

# Modify the head to fit the 7 classes
pretrained_vit.heads = nn.Linear(in_features=768, out_features=len(class_names)).to(device)

# Print model summary
summary(model=pretrained_vit, 
        input_size=(32, 3, 224, 224), # (batch_size, color_channels, height, width)
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
)


Using device: cuda


Downloading: "https://download.pytorch.org/models/vit_b_16-c867db91.pth" to /root/.cache/torch/hub/checkpoints/vit_b_16-c867db91.pth
100%|██████████| 330M/330M [00:01<00:00, 181MB/s]


Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
VisionTransformer (VisionTransformer)                        [32, 3, 224, 224]    [32, 7]              768                  Partial
├─Conv2d (conv_proj)                                         [32, 3, 224, 224]    [32, 768, 14, 14]    (590,592)            False
├─Encoder (encoder)                                          [32, 197, 768]       [32, 197, 768]       151,296              False
│    └─Dropout (dropout)                                     [32, 197, 768]       [32, 197, 768]       --                   --
│    └─Sequential (layers)                                   [32, 197, 768]       [32, 197, 768]       --                   False
│    │    └─EncoderBlock (encoder_layer_0)                   [32, 197, 768]       [32, 197, 768]       (7,087,872)          False
│    │    └─EncoderBlock (encoder_layer_1)                   [32, 197, 768]       [32, 

In [19]:
train_dir = '/kaggle/working/HAM10000_split/train'
test_dir = '/kaggle/working/HAM10000_split/test'


In [20]:
# Get automatic transforms from pretrained ViT weights
pretrained_vit_transforms = pretrained_vit_weights.transforms()
print(pretrained_vit_transforms)


ImageClassification(
    crop_size=[224]
    resize_size=[256]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BILINEAR
)


In [21]:
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

NUM_WORKERS = os.cpu_count()

def create_dataloaders(
    train_dir: str, 
    test_dir: str, 
    transform: transforms.Compose, 
    batch_size: int, 
    num_workers: int=NUM_WORKERS
):

    # Use ImageFolder to create dataset(s)
    train_data = datasets.ImageFolder(train_dir, transform=transform)
    test_data = datasets.ImageFolder(test_dir, transform=transform)

    # Get class names
    class_names = train_data.classes

    # Turn images into data loaders
    train_dataloader = DataLoader(
        train_data,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        pin_memory=True,
    )
    test_dataloader = DataLoader(
        test_data,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        pin_memory=True,
    )

    return train_dataloader, test_dataloader, class_names


# Update path for your Kaggle environment
train_dir = '/kaggle/working/HAM10000_split/train'  # Adjust path based on your dataset location
test_dir = '/kaggle/working/HAM10000_split/test'    # Adjust path based on your dataset location

# Define transformations (using ViT's pretrained weights transform)
pretrained_vit_transforms = pretrained_vit_weights.transforms()

# Setup dataloaders for ViT with the correct transformations
train_dataloader_pretrained, test_dataloader_pretrained, class_names = create_dataloaders(
    train_dir=train_dir,
    test_dir=test_dir,
    transform=pretrained_vit_transforms,
    batch_size=64  # Adjust the batch size as per your setup
)

# Optional: Print out class names and dataloader sizes
print(f"Class names: {class_names}")
print(f"Training dataloader size: {len(train_dataloader_pretrained)}")
print(f"Testing dataloader size: {len(test_dataloader_pretrained)}")


Class names: ['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'vasc']
Training dataloader size: 875
Testing dataloader size: 24


In [22]:
import torch
from tqdm.auto import tqdm
from typing import Dict, List, Tuple

# 1. Train Step
def train_step(model: torch.nn.Module, 
               dataloader: torch.utils.data.DataLoader, 
               loss_fn: torch.nn.Module, 
               optimizer: torch.optim.Optimizer,
               device: torch.device) -> Tuple[float, float]:
    """Trains a PyTorch model for a single epoch."""
    model.train()  # Set model to training mode
    train_loss, train_acc = 0, 0  # Initialize loss and accuracy

    # Loop through data loader data batches
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)  # Send data to device

        # 1. Forward pass
        y_pred = model(X)

        # 2. Calculate and accumulate loss
        loss = loss_fn(y_pred, y)
        train_loss += loss.item()

        # 3. Optimizer zero grad
        optimizer.zero_grad()

        # 4. Loss backward
        loss.backward()

        # 5. Optimizer step
        optimizer.step()

        # Calculate and accumulate accuracy
        y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
        train_acc += (y_pred_class == y).sum().item() / len(y_pred)

    # Return average loss and accuracy for the epoch
    train_loss = train_loss / len(dataloader)
    train_acc = train_acc / len(dataloader)
    return train_loss, train_acc

# 2. Test Step
def test_step(model: torch.nn.Module, 
              dataloader: torch.utils.data.DataLoader, 
              loss_fn: torch.nn.Module,
              device: torch.device) -> Tuple[float, float]:
    """Tests a PyTorch model for a single epoch."""
    model.eval()  # Set model to evaluation mode
    test_loss, test_acc = 0, 0  # Initialize loss and accuracy

    # Disable gradient calculations for inference
    with torch.inference_mode():
        for batch, (X, y) in enumerate(dataloader):
            X, y = X.to(device), y.to(device)  # Send data to device

            # 1. Forward pass
            test_pred_logits = model(X)

            # 2. Calculate and accumulate loss
            loss = loss_fn(test_pred_logits, y)
            test_loss += loss.item()

            # Calculate and accumulate accuracy
            test_pred_labels = test_pred_logits.argmax(dim=1)
            test_acc += ((test_pred_labels == y).sum().item() / len(test_pred_labels))

    # Return average loss and accuracy for the epoch
    test_loss = test_loss / len(dataloader)
    test_acc = test_acc / len(dataloader)
    return test_loss, test_acc

# 3. Train and Test Loop
def train(model: torch.nn.Module, 
          train_dataloader: torch.utils.data.DataLoader, 
          test_dataloader: torch.utils.data.DataLoader, 
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module,
          epochs: int,
          device: torch.device) -> Dict[str, List]:
    """Trains and tests a PyTorch model."""
    results = {
        "train_loss": [],
        "train_acc": [],
        "test_loss": [],
        "test_acc": []
    }
    
    model.to(device)  # Send model to target device (GPU/CPU)

    # Loop through training and testing steps for a number of epochs
    for epoch in tqdm(range(epochs)):
        # Training step
        train_loss, train_acc = train_step(model=model,
                                           dataloader=train_dataloader,
                                           loss_fn=loss_fn,
                                           optimizer=optimizer,
                                           device=device)
        
        # Testing step
        test_loss, test_acc = test_step(model=model,
                                        dataloader=test_dataloader,
                                        loss_fn=loss_fn,
                                        device=device)

        # Print metrics for the current epoch
        print(
            f"Epoch: {epoch+1} | "
            f"train_loss: {train_loss:.4f} | "
            f"train_acc: {train_acc:.4f} | "
            f"test_loss: {test_loss:.4f} | "
            f"test_acc: {test_acc:.4f}"
        )

        # Store results for each epoch
        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)

    return results


In [23]:
# Create optimizer and loss function
optimizer = torch.optim.Adam(params=pretrained_vit.parameters(), lr=1e-3)
loss_fn = torch.nn.CrossEntropyLoss()

# Train the model
pretrained_vit_results = train(
    model=pretrained_vit,
    train_dataloader=train_dataloader_pretrained,
    test_dataloader=test_dataloader_pretrained,
    optimizer=optimizer,
    loss_fn=loss_fn,
    epochs=30,
    device=device
)


  0%|          | 0/30 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 1.0096 | train_acc: 0.6360 | test_loss: 0.6706 | test_acc: 0.7527
Epoch: 2 | train_loss: 0.8168 | train_acc: 0.7065 | test_loss: 0.6348 | test_acc: 0.7769
Epoch: 3 | train_loss: 0.7583 | train_acc: 0.7275 | test_loss: 0.6406 | test_acc: 0.7796
Epoch: 4 | train_loss: 0.7255 | train_acc: 0.7384 | test_loss: 0.6174 | test_acc: 0.7841
Epoch: 5 | train_loss: 0.7028 | train_acc: 0.7486 | test_loss: 0.6200 | test_acc: 0.7959
Epoch: 6 | train_loss: 0.6884 | train_acc: 0.7511 | test_loss: 0.6263 | test_acc: 0.7822
Epoch: 7 | train_loss: 0.6751 | train_acc: 0.7575 | test_loss: 0.6189 | test_acc: 0.7958
Epoch: 8 | train_loss: 0.6664 | train_acc: 0.7599 | test_loss: 0.6129 | test_acc: 0.7789
Epoch: 9 | train_loss: 0.6583 | train_acc: 0.7631 | test_loss: 0.6011 | test_acc: 0.7906
Epoch: 10 | train_loss: 0.6527 | train_acc: 0.7651 | test_loss: 0.6120 | test_acc: 0.7855
Epoch: 11 | train_loss: 0.6456 | train_acc: 0.7664 | test_loss: 0.6191 | test_acc: 0.7932
Epoch: 12 | train_l

In [24]:
# Define model save path
model_save_path = "pretrained_vit.pth"

# Save the trained model's state_dict
torch.save(pretrained_vit.state_dict(), model_save_path)


In [25]:
# Load the trained model
loaded_model = pretrained_vit  # Reinitialize model architecture

# Load the saved state dictionary
loaded_model.load_state_dict(torch.load(model_save_path, map_location=device))

# Move model to the correct device
loaded_model.to(device)
loaded_model.eval()  # Set to evaluation mode


<ipython-input-25-7bc92fc931b5>:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loaded_model.load_state_dict(torch.load(model_save_path, map_location=device))


VisionTransformer(
  (conv_proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
  (encoder): Encoder(
    (dropout): Dropout(p=0.0, inplace=False)
    (layers): Sequential(
      (encoder_layer_0): EncoderBlock(
        (ln_1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (self_attention): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
        )
        (dropout): Dropout(p=0.0, inplace=False)
        (ln_2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (mlp): MLPBlock(
          (0): Linear(in_features=768, out_features=3072, bias=True)
          (1): GELU(approximate='none')
          (2): Dropout(p=0.0, inplace=False)
          (3): Linear(in_features=3072, out_features=768, bias=True)
          (4): Dropout(p=0.0, inplace=False)
        )
      )
      (encoder_layer_1): EncoderBlock(
        (ln_1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (self_a